In [1]:
from theano.sandbox import cuda

In [1]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function
from keras.layers import Embedding

Using TensorFlow backend.


In [2]:
#path = "data/ml-20m/"
path = "/mnt/data/ml-latest-small/"
model_path = '/mnt/models/ml-latest-small/'
if not os.path.exists(model_path): os.mkdir(model_path)
batch_size=64

## Set up data

We're working with the movielens data, which contains one rating per row, like this:

In [23]:
ratings = pd.read_csv(path+'ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [24]:
movie_names = pd.read_csv(path + 'movies.csv')
movie_names.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [25]:
movies = ratings.movieId.unique()
users = ratings.userId.unique()
movie2idx = {movie_id:idx for idx, movie_id in enumerate(movies)}
user2idx = {user_id:idx for idx, user_id in enumerate(users)}

In [26]:
ratings['new_user_id'] = ratings.userId.apply(lambda x: user2idx[x])
ratings['new_movie_id'] = ratings.movieId.apply(lambda x: movie2idx[x])

In [27]:
n_movies = ratings.movieId.nunique()
n_users = ratings.userId.nunique()

## Split into training and validation data

In [28]:
msk = (np.random.rand(len(ratings)) < 0.8)
trn = ratings[msk]
val = ratings[~msk]

## Config

In [29]:
n_factors = 50

## Fit Dot Product Model

In [11]:
movie_in = Input(shape=(1,), dtype='int64', name='movie_in')
user_in = Input(shape=(1,), dtype='int64', name='user_in')
movie_emb = Embedding(n_movies, n_factors, input_length=1, W_regularizer=l2(1e-4))(movie_in)
user_emb = Embedding(n_movies, n_factors, input_length=1, W_regularizer=l2(1e-4))(user_in)

In [12]:
x = merge([user_emb, movie_emb], mode='dot')
x = Flatten()(x)
model = Model([user_in, movie_in], x)
model.compile(optimizer=Adam(0.01), loss='mse')

In [49]:
model.fit([trn.new_user_id, trn.new_movie_id], trn.rating, nb_epoch=10, batch_size=64,
          validation_data=([val.new_user_id, val.new_movie_id], val.rating))

Train on 80186 samples, validate on 19818 samples
Epoch 1/10
80186/80186 [==============================] - 5s - loss: 5.0599 - val_loss: 3.4525
Epoch 2/10
80186/80186 [==============================] - 5s - loss: 3.2580 - val_loss: 3.6369
Epoch 3/10
80186/80186 [==============================] - 5s - loss: 3.1716 - val_loss: 3.6626
Epoch 4/10
80186/80186 [==============================] - 5s - loss: 3.0621 - val_loss: 3.6256
Epoch 5/10
80186/80186 [==============================] - 5s - loss: 2.9817 - val_loss: 3.6040
Epoch 6/10
80186/80186 [==============================] - 6s - loss: 2.8927 - val_loss: 3.5577
Epoch 7/10
80186/80186 [==============================] - 5s - loss: 2.8284 - val_loss: 3.5328
Epoch 8/10
80186/80186 [==============================] - 5s - loss: 2.7717 - val_loss: 3.4825
Epoch 9/10
80186/80186 [==============================] - 5s - loss: 2.7378 - val_loss: 3.4557
Epoch 10/10
80186/80186 [==============================] - 5s - loss: 2.6979 - val_loss: 3.4371

## Bias Model

In [34]:
user_in = Input(shape=(1,), dtype='int64', name='user_in')
user_emb = Embedding(input_dim=n_users, output_dim=n_factors, input_length=1, W_regularizer=l2(1e-4))(user_in)
movie_in = Input(shape=(1,), dtype='int64', name='movie_in')
movie_emb = Embedding(input_dim=n_movies, output_dim=n_factors, input_length=1, W_regularizer=l2(1e-4))(movie_in)

In [35]:
movie_bias = Embedding(input_dim=n_movies, output_dim=1, input_length=1)(movie_in)
user_bias = Embedding(input_dim=n_users, output_dim=1, input_length=1)(user_in)

In [36]:
x = merge([user_emb, movie_emb], mode='dot')
x = merge([x, user_bias], mode='sum')
x = merge([x, movie_bias], mode='sum')
x = Flatten()(x)

In [37]:
model = Model([user_in, movie_in], x)
model.compile(optimizer=Adam(0.001), loss='mse')

In [38]:
model.fit([trn.new_user_id, trn.new_movie_id], trn.rating, nb_epoch=1,
          batch_size=64, validation_data=([val.new_user_id, val.new_movie_id], val.rating))

Train on 80170 samples, validate on 19834 samples
Epoch 1/1
80170/80170 [==============================] - 5s - loss: 8.7955 - val_loss: 3.4770


In [39]:
model.optimizer.lr = 0.01
model.fit([trn.new_user_id, trn.new_movie_id], trn.rating, nb_epoch=1,
          batch_size=64, validation_data=([val.new_user_id, val.new_movie_id], val.rating))

Train on 80170 samples, validate on 19834 samples
Epoch 1/1
80170/80170 [==============================] - 5s - loss: 2.5825 - val_loss: 2.2996


In [46]:
model.optimizer.lr = 0.001
model.fit([trn.new_user_id, trn.new_movie_id], trn.rating, nb_epoch=5,
          batch_size=64, validation_data=([val.new_user_id, val.new_movie_id], val.rating))

Train on 80170 samples, validate on 19834 samples
Epoch 1/5
80170/80170 [==============================] - 5s - loss: 0.5682 - val_loss: 1.0244
Epoch 2/5
80170/80170 [==============================] - 5s - loss: 0.5624 - val_loss: 1.0230
Epoch 3/5
80170/80170 [==============================] - 5s - loss: 0.5571 - val_loss: 1.0214
Epoch 4/5
80170/80170 [==============================] - 5s - loss: 0.5529 - val_loss: 1.0205
Epoch 5/5
80170/80170 [==============================] - 5s - loss: 0.5495 - val_loss: 1.0211


## Basic Neural Network 

In [66]:
user_in = Input(shape=(1,), dtype='int64', name='user_in')
user_emb = Embedding(n_users, n_factors, input_length=1)(user_in)
movie_in = Input(shape=(1,), dtype='int64', name='movie_in')
movie_emb = Embedding(n_movies, n_factors, input_length=1)(movie_in)

In [67]:
x1 = Dense(50, activation='relu')(user_emb)
x2 = Dense(50, activation='relu')(movie_emb)
x = merge([x1, x2], mode='concat')
x = Flatten()(x)
x = Dense(1, activation='relu')(x)

In [53]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
user_in (InputLayer)             (None, 1)             0                                            
____________________________________________________________________________________________________
movie_in (InputLayer)            (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_19 (Embedding)         (None, 1, 50)         33550       user_in[0][0]                    
____________________________________________________________________________________________________
embedding_20 (Embedding)         (None, 1, 50)         453300      movie_in[0][0]                   
___________________________________________________________________________________________

In [68]:
model = Model([user_in, movie_in], x)
model.compile(optimizer=Adam(lr=0.001), loss='mse')

In [69]:
model.fit([trn.new_user_id, trn.new_movie_id], trn.rating, nb_epoch=1,
         batch_size=64, validation_data=([val.new_user_id, val.new_movie_id], val.rating))

Train on 80170 samples, validate on 19834 samples
Epoch 1/1
80170/80170 [==============================] - 6s - loss: 1.4875 - val_loss: 0.8169
